In [1]:
%matplotlib inline
import os
import sys
import gc
from tqdm.notebook import tqdm

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from commons.configuration_manager import ConfigurationManager
from src.learning.training.training_transformer import TrainingTransformer
from src.learning.training.label_collector import LabelCollector
from src.learning.training.training_file_reader import TrainingFileReader
from src.learning.models import create_cnn, create_mlp, create_multi_model

In [2]:
config_manager = ConfigurationManager()
config = config_manager.config

reader = TrainingFileReader(path_to_training='../../training/laps/')
transformer = TrainingTransformer(config)
collector = LabelCollector()

path_to_training = '../../training/laps/'

def plot_stuff(title, plot_elems, figsize=(18, 10)):
    fig=plt.figure(figsize=figsize)
    plt.title(title)
    #plt.ylabel('dunno')
    plt.xlabel('Count')
    
    for plot_elem in plot_elems:
        plt.plot(plot_elem['data'], label=plot_elem['label'], alpha=plot_elem['alpha'])

    plt.grid(axis='y')
    plt.legend(loc='best')
    plt.show()
    
# axis=2 for frames, axis=0 for telem and diffs
def memory_creator(instance, memory, length=4, interval=2, axis=2):
    memory.append(instance)
    
    near_memory = memory[::-interval]
    if len(near_memory) < length:
        return None
    
    if len(memory) >= length * interval:
        memory.pop(0)
        
    return np.concatenate(near_memory, axis=axis)


def read_stored_data_for_training(filename):
    df = reader.read_telemetry_as_csv(filename + '.csv')
    telemetry = collector.collect_numeric_inputs(df)
    diffs = collector.collect_expert_labels(df)

    frames = reader.read_training_video(filename + '.avi')
    resized_frames_np = transformer.resize_video_for_training(frames)

    return resized_frames_np, telemetry, diffs


def read_stored_video_for_display(filename):
    return reader.read_training_video(filename + '.avi')
    
    
def prep_full_mem_datas(frames, telemetry, diffs, length, interval):    
    mem_slice_frames = []
    mem_slice_telemetry = []
    
    mem_frames = []
    for frame in frames:
        mem_frame = memory_creator(frame, mem_slice_frames, length=length, interval=interval, axis=2)

        if mem_frame is not None:
            mem_frames.append(mem_frame)
    mem_frames_np = np.array(mem_frames)
    
    mem_telemetry = []
    for index, telem in telemetry.iterrows():
        mem_telem = memory_creator(telem, mem_slice_telemetry, length=length, interval=interval, axis=0)

        if mem_telem is not None:
            mem_telemetry.append(mem_telem)
    mem_telemetry_np = np.array(mem_telemetry)
    
    mem_diffs = diffs
    len_diff = diffs.shape[0] - mem_frames_np.shape[0]
    if len_diff > 0:
        mem_diffs = mem_diffs.iloc[len_diff:]
    mem_diffs_np = mem_diffs.to_numpy()
    
    assert mem_frames_np.shape[0] == mem_telemetry_np.shape[0] == mem_diffs_np.shape[0], "Lengths differ!"
    return mem_frames_np, mem_telemetry_np, mem_diffs_np

In [ ]:
filename = 'lap_7_2020_01_24.csv'

df = pd.read_csv(path_to_training + filename)
#df = df[8000:25000]

print(df.shape)
print(df.columns.values)

In [ ]:
steering = [{'data': df.d_steering_expert, 'label':'expert diff', 'alpha':1.0}, 
         {'data': df.cs, 'label':'steering value', 'alpha':0.5}]

plot_stuff('Steering', steering)

In [ ]:
steering = [{'data': df.d_throttle_expert, 'label':'expert diff', 'alpha':1.0}, 
         {'data': df.ct, 'label':'throttle value', 'alpha':0.5}]

plot_stuff('Throttle', steering)

In [3]:
filenames = ['lap_5_2020_01_24', 'lap_6_2020_01_24', 'lap_7_2020_01_24']
experiments = [(1, 1), (4, 1), (8, 1), (16, 1)]
epochs = 20
batch_size = 32
verbose = 0

mem_frames, mem_telems, mem_diffs = prep_full_mem_datas(*read_stored_data_for_training(filenames[0]), *experiments[1])
frames_train, frames_test, telemetry_train, telemetry_test, diffs_train, diffs_test = train_test_split(mem_frames, mem_telems, mem_diffs, test_size=0.2)

mlp = create_mlp(input_shape=telemetry_train[0].shape)
cnn = create_cnn(input_shape=frames_train[0].shape)
multi = create_multi_model(mlp, cnn)

multi.fit([telemetry_train, frames_train], diffs_train,
            validation_data=([telemetry_test, frames_test], diffs_test),
            epochs=epochs, batch_size=batch_size, verbose=verbose)

In [4]:
import pygame
import cv2

display_frames = read_stored_video_for_display(filenames[0])
display_frames = display_frames[(display_frames.shape[0] - mem_frames.shape[0]):]

window_width = 500
window_height = 200

black = (0, 0, 0)
red = (255, 0, 0)
green = (0, 255, 0)
blue = (0, 0, 255)

pygame.init()
font = pygame.font.SysFont('Roboto', 20)

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [7]:
pygame.display.init()
pygame.display.set_caption("Prediction viewer")
screen = pygame.display.set_mode((window_width, window_height))

try:
    for i in range(0, display_frames.shape[0]):
        frame = display_frames[i]
        screen.fill(black)
        
        converted_telem = mem_telems[i][np.newaxis, :]
        converted_frame = mem_frames[i].astype(np.float32)[np.newaxis, :]
        
        # gear, steering, throttle, braking
        predictions = multi.predict([converted_telem, converted_frame])[0]
        #print(mem_diffs[i])
        #print(predictions)
        
        prediction_text = "{}".format(predictions[1])
        prediction_texture = font.render(prediction_text, True, green)
        screen.blit(prediction_texture, (200, window_height - 25))
        
        frame = np.rot90(frame)
        surface = pygame.surfarray.make_surface(frame)
        
        x = (window_width - frame.shape[0]) // 2
        y = 0
        screen.blit(surface, (x, y))
        pygame.display.update()

        for event in pygame.event.get():
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    sys.exit(0)
    pygame.display.quit()
    
except (KeyboardInterrupt, SystemExit) as ex:
    pygame.display.quit()